用 MinimumEigenOptimizer 解决 qubo

In [1]:
from qiskit_optimization.problems import QuadraticProgram
qp = QuadraticProgram()
qp.binary_var('x')
qp.binary_var('y')
qp.binary_var('z')
qp.minimize(linear = {'y':-1}, quadratic = {('x','y'):2, ('z','y'):-4})
qp.linear_constraint(linear = {'x':1, 'y':2, 'z':3},
sense ="<=", rhs = 5)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - y + [ 4 x*y - 8 y*z ]/2
Subject To
 c0: x + 2 y + 3 z <= 5

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



In [4]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import NumPyMinimumEigensolver
np_solver = NumPyMinimumEigensolver()
np_optimizer = MinimumEigenOptimizer(np_solver)
result = np_optimizer.solve(qp)
print(result)

fval=-5.0, x=0.0, y=1.0, z=1.0, status=SUCCESS


C:\Users\Newt\AppData\Local\Temp\ipykernel_32164\3122343473.py:3: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  np_solver = NumPyMinimumEigensolver()


qaoa

In [6]:
from qiskit import Aer
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator"),
shots = 1024)
qaoa = QAOA(optimizer = COBYLA(),
quantum_instance=quantum_instance, reps = 1)
qaoa_optimizer = MinimumEigenOptimizer(qaoa)
result = qaoa_optimizer.solve(qp)
print(result)

C:\Users\Newt\AppData\Local\Temp\ipykernel_32164\4112302810.py:5: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator"),
C:\Users\Newt\AppData\Local\Temp\ipykernel_32164\4112302810.py:7: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.qaoa.QAOA`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.QAOA``. See https://qisk.it/algo_migration for a migration guide.
  qaoa = QAOA(optimizer = COBYLA(),


fval=-5.0, x=0.0, y=1.0, z=1.0, status=SUCCESS


In [8]:
print('Variable order:', [var.name for var in result.variables])
for s in result.samples:
    print(s)

Variable order: ['x', 'y', 'z']
SolutionSample(x=array([0., 1., 1.]), fval=-5.0, probability=0.1318359375, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 1., 0.]), fval=-1.0, probability=0.1005859375, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 0., 1.]), fval=0.0, probability=0.10546875, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 0., 1.]), fval=0.0, probability=0.1015625, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 0., 0.]), fval=0.0, probability=0.1455078125, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([0., 0., 0.]), fval=0.0, probability=0.140625, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 1., 0.]), fval=1.0, probability=0.111328125, status=<OptimizationResultStatus.SUCCESS: 0>)
SolutionSample(x=array([1., 1., 1.]), fval=-3.0, probability=0.1630859375, status=<OptimizationResultStatus.INFEASIBLE: 2>)


In [9]:
print(result.min_eigen_solver_result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 30,
    'eigenstate': {   '000000': 0.10364452469860624,
                      '000001': 0.11267347735824966,
                      '000010': 0.08838834764831845,
                      '000011': 0.05412658773652741,
                      '000100': 0.08838834764831845,
                      '000101': 0.11692679333668567,
                      '000110': 0.0625,
                      '000111': 0.23593232610221093,
                      '001000': 0.06987712429686843,
                      '001001': 0.07654655446197431,
                      '001011': 0.13975424859373686,
                      '001100': 0.11267347735824966,
                      '001101': 0.10364452469860624,
                      '001110': 0.25,
                      '001111': 0.15934435979977452,
                      '010000': 0.15625,
                      '010001': 0.07654655446197431,
                      '010010': 0.14986973510352247,
                 

In [10]:
from qiskit_optimization.converters import QuadraticProgramToQubo
qp_to_qubo = QuadraticProgramToQubo()
qubo = qp_to_qubo.convert(qp)
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 80 x - 161 y - 240 z - 80 c0@int_slack@0 - 160 c0@int_slack@1
      - 160 c0@int_slack@2 + [ 16 x^2 + 68 x*y + 96 x*z + 32 x*c0@int_slack@0
      + 64 x*c0@int_slack@1 + 64 x*c0@int_slack@2 + 64 y^2 + 184 y*z
      + 64 y*c0@int_slack@0 + 128 y*c0@int_slack@1 + 128 y*c0@int_slack@2
      + 144 z^2 + 96 z*c0@int_slack@0 + 192 z*c0@int_slack@1
      + 192 z*c0@int_slack@2 + 16 c0@int_slack@0^2
      + 64 c0@int_slack@0*c0@int_slack@1 + 64 c0@int_slack@0*c0@int_slack@2
      + 64 c0@int_slack@1^2 + 128 c0@int_slack@1*c0@int_slack@2
      + 64 c0@int_slack@2^2 ]/2 + 200
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1
 0 <= c0@int_slack@0 <= 1
 0 <= c0@int_slack@1 <= 1
 0 <= c0@int_slack@2 <= 1

Binaries
 x y z c0@int_slack@0 c0@int_slack@1 c0@int_slack@2
End



ising

In [17]:
H1, offset = qubo.to_ising()

print("The Hamiltonian is", H1)
print("The constant term is", offset)

The Hamiltonian is -7.0 * IIIIZI
- 4.5 * IIIIIZ
- 11.0 * IIIZII
- 4.0 * IIZIII
- 8.0 * IZIIII
- 8.0 * ZIIIII
+ 8.5 * IIIIZZ
+ 12.0 * IIIZIZ
+ 23.0 * IIIZZI
+ 4.0 * IIZIIZ
+ 8.0 * IIZIZI
+ 12.0 * IIZZII
+ 8.0 * IZIIIZ
+ 16.0 * IZIIZI
+ 24.0 * IZIZII
+ 8.0 * IZZIII
+ 8.0 * ZIIIIZ
+ 16.0 * ZIIIZI
+ 24.0 * ZIIZII
+ 8.0 * ZIZIII
+ 16.0 * ZZIIII
The constant term is 47.0


In [20]:
from qiskit_optimization.runtime import QAOAClient
from qiskit import IBMQ
provider = IBMQ.load_account()
qaoa_client = QAOAClient(provider=provider,backend=provider.get_backend("ibm_lagos"), reps=1)
qaoa = MinimumEigenOptimizer(qaoa_client)
result = qaoa.solve(qp)
print(result)

ibmqfactory.load_account:WARNING:2023-11-01 16:39:55,567: Credentials are already in use. The existing account in the session will be replaced.


RequestsApiError: '404 Client Error: Not Found for url: https://api.quantum-computing.ibm.com/runtime/jobs. {"errors":[{"code":1211,"message":"Program not found.","solution":"Upload program or use a different program name.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]}'